# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd9d435b370>
├── 'a' --> tensor([[-1.3007,  0.7796, -2.2928],
│                   [-0.4884,  0.5153, -1.7789]])
└── 'x' --> <FastTreeValue 0x7fda78306190>
    └── 'c' --> tensor([[-1.2972, -1.3576,  1.4951,  0.6746],
                        [-0.2503, -0.4120,  1.8680, -0.1202],
                        [-0.0511, -0.1759,  0.9793,  1.0093]])

In [4]:
t.a

tensor([[-1.3007,  0.7796, -2.2928],
        [-0.4884,  0.5153, -1.7789]])

In [5]:
%timeit t.a

75.8 ns ± 0.584 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd9d435b370>
├── 'a' --> tensor([[-1.5389e+00, -1.4651e+00, -2.1312e-04],
│                   [ 5.1682e-01,  3.5543e-01, -6.7369e-02]])
└── 'x' --> <FastTreeValue 0x7fda78306190>
    └── 'c' --> tensor([[-1.2972, -1.3576,  1.4951,  0.6746],
                        [-0.2503, -0.4120,  1.8680, -0.1202],
                        [-0.0511, -0.1759,  0.9793,  1.0093]])

In [7]:
%timeit t.a = new_value

78 ns ± 0.325 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.3007,  0.7796, -2.2928],
               [-0.4884,  0.5153, -1.7789]]),
    x: Batch(
           c: tensor([[-1.2972, -1.3576,  1.4951,  0.6746],
                      [-0.2503, -0.4120,  1.8680, -0.1202],
                      [-0.0511, -0.1759,  0.9793,  1.0093]]),
       ),
)

In [10]:
b.a

tensor([[-1.3007,  0.7796, -2.2928],
        [-0.4884,  0.5153, -1.7789]])

In [11]:
%timeit b.a

69.6 ns ± 0.434 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1301,  1.6840, -0.2957],
               [-1.0503,  0.7669, -0.0469]]),
    x: Batch(
           c: tensor([[-1.2972, -1.3576,  1.4951,  0.6746],
                      [-0.2503, -0.4120,  1.8680, -0.1202],
                      [-0.0511, -0.1759,  0.9793,  1.0093]]),
       ),
)

In [13]:
%timeit b.a = new_value

572 ns ± 3.19 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

985 ns ± 19.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.8 µs ± 41.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

164 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

161 µs ± 807 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd9cd5f5f70>
├── 'a' --> tensor([[[-1.3007,  0.7796, -2.2928],
│                    [-0.4884,  0.5153, -1.7789]],
│           
│                   [[-1.3007,  0.7796, -2.2928],
│                    [-0.4884,  0.5153, -1.7789]],
│           
│                   [[-1.3007,  0.7796, -2.2928],
│                    [-0.4884,  0.5153, -1.7789]],
│           
│                   [[-1.3007,  0.7796, -2.2928],
│                    [-0.4884,  0.5153, -1.7789]],
│           
│                   [[-1.3007,  0.7796, -2.2928],
│                    [-0.4884,  0.5153, -1.7789]],
│           
│                   [[-1.3007,  0.7796, -2.2928],
│                    [-0.4884,  0.5153, -1.7789]],
│           
│                   [[-1.3007,  0.7796, -2.2928],
│                    [-0.4884,  0.5153, -1.7789]],
│           
│                   [[-1.3007,  0.7796, -2.2928],
│                    [-0.4884,  0.5153, -1.7789]]])
└── 'x' --> <FastTreeValue 0x7fd9ccfd90a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

35.9 µs ± 141 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd9d3be66a0>
├── 'a' --> tensor([[-1.3007,  0.7796, -2.2928],
│                   [-0.4884,  0.5153, -1.7789],
│                   [-1.3007,  0.7796, -2.2928],
│                   [-0.4884,  0.5153, -1.7789],
│                   [-1.3007,  0.7796, -2.2928],
│                   [-0.4884,  0.5153, -1.7789],
│                   [-1.3007,  0.7796, -2.2928],
│                   [-0.4884,  0.5153, -1.7789],
│                   [-1.3007,  0.7796, -2.2928],
│                   [-0.4884,  0.5153, -1.7789],
│                   [-1.3007,  0.7796, -2.2928],
│                   [-0.4884,  0.5153, -1.7789],
│                   [-1.3007,  0.7796, -2.2928],
│                   [-0.4884,  0.5153, -1.7789],
│                   [-1.3007,  0.7796, -2.2928],
│                   [-0.4884,  0.5153, -1.7789]])
└── 'x' --> <FastTreeValue 0x7fd9d3be6df0>
    └── 'c' --> tensor([[-1.2972, -1.3576,  1.4951,  0.6746],
                        [-0.2503, -0.4120,  1.8680, -0.1202],
                 

In [23]:
%timeit t_cat(trees)

34.2 µs ± 173 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

63.1 µs ± 436 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.2972, -1.3576,  1.4951,  0.6746],
                       [-0.2503, -0.4120,  1.8680, -0.1202],
                       [-0.0511, -0.1759,  0.9793,  1.0093]],
              
                      [[-1.2972, -1.3576,  1.4951,  0.6746],
                       [-0.2503, -0.4120,  1.8680, -0.1202],
                       [-0.0511, -0.1759,  0.9793,  1.0093]],
              
                      [[-1.2972, -1.3576,  1.4951,  0.6746],
                       [-0.2503, -0.4120,  1.8680, -0.1202],
                       [-0.0511, -0.1759,  0.9793,  1.0093]],
              
                      [[-1.2972, -1.3576,  1.4951,  0.6746],
                       [-0.2503, -0.4120,  1.8680, -0.1202],
                       [-0.0511, -0.1759,  0.9793,  1.0093]],
              
                      [[-1.2972, -1.3576,  1.4951,  0.6746],
                       [-0.2503, -0.4120,  1.8680, -0.1202],
                       [-0.0511, -0.1759,  0.9793,  1.0093]],

In [26]:
%timeit Batch.stack(batches)

88.1 µs ± 499 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.2972, -1.3576,  1.4951,  0.6746],
                      [-0.2503, -0.4120,  1.8680, -0.1202],
                      [-0.0511, -0.1759,  0.9793,  1.0093],
                      [-1.2972, -1.3576,  1.4951,  0.6746],
                      [-0.2503, -0.4120,  1.8680, -0.1202],
                      [-0.0511, -0.1759,  0.9793,  1.0093],
                      [-1.2972, -1.3576,  1.4951,  0.6746],
                      [-0.2503, -0.4120,  1.8680, -0.1202],
                      [-0.0511, -0.1759,  0.9793,  1.0093],
                      [-1.2972, -1.3576,  1.4951,  0.6746],
                      [-0.2503, -0.4120,  1.8680, -0.1202],
                      [-0.0511, -0.1759,  0.9793,  1.0093],
                      [-1.2972, -1.3576,  1.4951,  0.6746],
                      [-0.2503, -0.4120,  1.8680, -0.1202],
                      [-0.0511, -0.1759,  0.9793,  1.0093],
                      [-1.2972, -1.3576,  1.4951,  0.6746],
                   

In [28]:
%timeit Batch.cat(batches)

163 µs ± 665 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

474 µs ± 204 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
